## 1. Import thư viện

In [1]:
import sys
import os
import numpy as np

# Thêm thư mục src vào sys.path để import các module tự định nghĩa
sys.path.append(os.path.abspath(os.path.join('..')))

from src import data_processing as dp

## 2. Quy trình tiền xử lý

**Giai đoạn 1: Làm Sạch & Chọn Lọc Dữ Liệu**
1.  **Xử lý cột định danh**:
    *   Loại bỏ cột `enrollee_id` vì không mang lại giá trị dự báo.
    *   Đối với cột `city`, thay vì sử dụng One-Hot Encoding (tạo ra quá nhiều cột) hoặc loại bỏ, ta sẽ **trích xuất phần số** của mã thành phố (ví dụ: `city_103` $\rightarrow$ `103`) để giữ lại thông tin địa lý dưới dạng biến số/phân loại.
2.  **Xử lý dữ liệu trùng lặp**: Các dòng bị trùng lặp hoàn toàn sẽ bị loại bỏ khỏi tập huấn luyện để tránh sai lệch trọng số mô hình. Bước này được bỏ qua đối với tập Test.

**Giai đoạn 2: Mã hóa & Xử lý Giá trị thiếu (Imputation)**

1.  **Tạo biến chỉ báo (Indicator Variables)**:
    *   Trước khi điền giá trị thiếu, ta tạo các cột mới (ví dụ: `education_level_is_missing`) để đánh dấu các vị trí bị thiếu dữ liệu.
    *   Điều này giúp mô hình học được "mẫu hình" của việc thiếu dữ liệu (như đã phân tích ở phần EDA, việc thiếu thông tin công ty là một tín hiệu quan trọng).

2.  **Mã hóa & Điền giá trị thiếu (Median Imputation)**:
    *   **Mã hóa:** Các biến phân loại được chuyển đổi sang dạng số (Ordinal Encoding) dựa trên bảng ánh xạ định trước.
    *   **Xử lý thiếu:** Các giá trị bị thiếu (NaN hoặc rỗng) được điền bằng giá trị **trung vị** của cột đó.
        *   *Lý do:* Trung vị ít bị ảnh hưởng bởi các giá trị ngoại lai hơn so với trung bình, đảm bảo tính ổn định cho dữ liệu.
    *   **Làm tròn:** Sau khi điền median, các cột vốn là biến phân loại sẽ được làm tròn về số nguyên gần nhất để đảm bảo tính nhất quán của dữ liệu.

In [2]:
def get_ordinal_mappings():
    """Định nghĩa ánh xạ ordinal cho các biến phân loại."""
    
    return {
        'experience': {
            '<1': 0, '>20': 21, 
            **{str(i): i for i in range(1, 21)}
        },
        'relevent_experience': {'No relevent experience': 0, 
                                'Has relevent experience': 1},
        'education_level': {'Primary School': 1, 
                            'High School': 2, 
                            'Graduate': 3, 
                            'Masters': 4, 
                            'Phd': 5},
        'last_new_job': {'never': 0, 
                         '1': 1, 
                         '2': 2, 
                         '3': 3, 
                         '4': 4, 
                         '>4': 5},
        'company_size': {
            '<10': 0, '10/49': 1, '50-99': 2, '100-500': 3,
            '500-999': 4, '1000-4999': 5, '5000-9999': 6, '10000+': 7
        },
        'gender': {'Female': 0, 'Male': 1, 'Other': 2},
        'enrolled_university': {'no_enrollment': 0, 
                                'Part time course': 1, 
                                'Full time course': 2},
        'major_discipline': {
            'No Major': 0, 'Humanities': 1, 'Arts': 2,
            'Business Degree': 3, 'STEM': 4, 'Other': 5
        },
        'company_type': {
            'NGO': 0, 'Public Sector': 1, 'Early Stage Startup': 2,
            'Funded Startup': 3, 'Pvt Ltd': 4, 'Other': 5
        }
    }

In [ ]:
def preprocess_pipeline(data, is_test_set=False):
    """
    Pipeline tiền xử lý:
    1. Giữ lại City (parse số).
    2. Chuyển đổi Categorical sang số (NaN nếu thiếu).
    3. Tạo cột Indicator cho giá trị thiếu.
    4. Điền giá trị thiếu bằng Median.
    """
    
    # 1. Loại bỏ enrollee_id
    data = dp.remove_columns(data, ['enrollee_id'])
    
    if not is_test_set:
        data = dp.remove_duplicates(data)
    
    mappings = get_ordinal_mappings()
    
    feature_names = []      # Tên các cột dữ liệu chính
    columns_data = []       # Dữ liệu dạng số của các cột
    cat_indices = []        # Chỉ mục các cột là categorical (để làm tròn sau này)
    indicator_cols = {}     # Các cột đánh dấu missing
    
    col_idx = 0
    
    for name in data.dtype.names:
        if name == 'target':
            continue
            
        col = data[name]
        feature_names.append(name)
        
        # --- A. Phát hiện giá trị thiếu ---
        if col.dtype.kind in ('O', 'U', 'S'):  # Kiểu chuỗi
            is_missing = (col == '').astype(int)
        else:  # Kiểu số
            is_missing = np.isnan(col).astype(int)
            
        # Nếu cột có giá trị thiếu, tạo cột indicator
        if is_missing.any():
            indicator_cols[f"{name}_is_missing"] = is_missing
        
        # --- B. Xử lý & Mã hóa từng loại cột ---
        
        # Trường hợp 1: Cột có trong mapping
        if name in mappings:
            cat_indices.append(col_idx)
            numeric_col = np.full(len(col), np.nan, dtype=float)
            # Map giá trị string sang số, giữ lại NaN nếu không map được (missing)
            for val, code in mappings[name].items():
                numeric_col[col == val] = code
            columns_data.append(numeric_col)
            
        # Trường hợp 2: Cột City (Xử lý đặc biệt: cắt chuỗi lấy số)
        elif name == 'city':
            # city_103 -> 103.0
            city_num = np.array([
                float(c.split('_')[1]) if c and '_' in c else np.nan
                for c in col
            ])
            columns_data.append(city_num)
            
        # Trường hợp 3: Cột số thông thường (training_hours, city_development_index)
        elif np.issubdtype(col.dtype, np.number):
            columns_data.append(col.astype(float))
            
        else:
            print(f"Cảnh báo: Bỏ qua cột không hỗ trợ: {name}")
            feature_names.pop()
            if f"{name}_is_missing" in indicator_cols:
                del indicator_cols[f"{name}_is_missing"]
            continue
            
        col_idx += 1
    
    # Gộp tất cả thành ma trận X để tính toán vector hóa
    X = np.column_stack(columns_data)
    
    # --- C. Điền giá trị thiếu bằng Median (Median Imputation) ---
    medians = np.nanmedian(X, axis=0)
    
    # Tìm vị trí các ô NaN và điền bằng median của cột tương ứng
    rows_nan, cols_nan = np.where(np.isnan(X))
    X[rows_nan, cols_nan] = medians[cols_nan]
    
    # --- D. Làm tròn các cột Categorical ---
    # Vì median có thể là số lẻ (vd: 1.5), cần làm tròn về int cho các biến phân loại
    for idx in cat_indices:
        X[:, idx] = np.round(X[:, idx])
    
    # --- E. Tái tạo mảng cấu trúc (Structured Array) ---
    # Định nghĩa dtype cho mảng kết quả
    new_dtype = []
    
    # 1. Các cột đặc trưng chính
    for i, name in enumerate(feature_names):
        if i in cat_indices or name == 'city':
            new_dtype.append((name, 'i4')) # Integer
        else:
            new_dtype.append((name, 'f8')) # Float
            
    # 2. Các cột indicator (is_missing)
    for ind_name in indicator_cols:
        new_dtype.append((ind_name, 'i4'))
        
    # 3. Cột target (nếu có)
    if 'target' in data.dtype.names:
        new_dtype.append(('target', 'i4')) # Target là 0 hoặc 1 nên để int
    
    # Tạo mảng kết quả
    result = np.zeros(X.shape[0], dtype=new_dtype)
    
    # Gán dữ liệu vào mảng kết quả
    for i, name in enumerate(feature_names):
        result[name] = X[:, i]
        
    for name, arr in indicator_cols.items():
        result[name] = arr
        
    if 'target' in data.dtype.names:
        result['target'] = data['target']
    
    return result

## 3. Áp dụng pipeline và xuất dữ liệu

Bây giờ chúng ta sẽ áp dụng pipeline trên cho cả hai tệp `aug_train.csv` và `aug_test.csv`, sau đó lưu kết quả đã xử lý vào thư mục `data/processed`.

In [4]:
def process_and_save(input_filename, output_filename, is_test_set=False):
    """Tải, xử lý và lưu một tệp dữ liệu."""
    
    # Tải dữ liệu
    raw_filepath = f'../data/raw/{input_filename}'
    raw_data = dp.load_data(raw_filepath)
    print(f"Đã tải {input_filename} với shape: {raw_data.shape}")
    
    # Áp dụng pipeline
    processed_data = preprocess_pipeline(raw_data, is_test_set=is_test_set)
    print(f"Tiền xử lý hoàn tất cho {input_filename}. Shape mới: {processed_data.shape}")
    
    # Lưu dữ liệu
    output_dir = '../data/processed'
    os.makedirs(output_dir, exist_ok=True)
    output_filepath = os.path.join(output_dir, output_filename)
    
    header = ','.join(processed_data.dtype.names)
    
    # Tạo format string động: %d cho số nguyên, %g cho số thực
    fmt_list = []
    for name in processed_data.dtype.names:
        if processed_data.dtype[name].kind in ('i', 'u'): # Integer/Unsigned
            fmt_list.append('%d')
        else:
            fmt_list.append('%g') # General format (tự bỏ số 0 thừa)

    np.savetxt(output_filepath, processed_data, delimiter=',', fmt=fmt_list, header=header, comments='')
    print(f"Đã lưu dữ liệu đã xử lý vào: {output_filepath}")
    print("-" * 50)

In [5]:
# Xử lý tập train
process_and_save('aug_train.csv', 'aug_train_processed.csv', is_test_set=False)

# Xử lý tập test
process_and_save('aug_test.csv', 'aug_test_processed.csv', is_test_set=True)

Đã tải aug_train.csv với shape: (19158,)
Tiền xử lý hoàn tất cho aug_train.csv. Shape mới: (19109,)
Đã lưu dữ liệu đã xử lý vào: ../data/processed/aug_train_processed.csv
--------------------------------------------------
Đã tải aug_test.csv với shape: (2129,)
Tiền xử lý hoàn tất cho aug_test.csv. Shape mới: (2129,)
Đã lưu dữ liệu đã xử lý vào: ../data/processed/aug_test_processed.csv
--------------------------------------------------
